In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

from __future__ import print_function
from keras.models import Model
from keras.layers import Dense, Input
from keras.datasets import mnist
from keras.regularizers import l1
from keras.optimizers import Adam

In [ ]:
def plot_autoencoder_outputs(autoencoder, n, dims):
    decoded_imgs = autoencoder.predict(x_test)

    # n= number of example digits to show
    plt.figure(figsize=(10, 4.5))
    for i in range(n):
        # plot original image
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[i].reshape(*dims))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == n/2:
            ax.set_title('Original Images')

        # plot reconstruction 
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(*dims))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == n/2:
            ax.set_title('Reconstructed Images')
    plt.show()

def plot_loss(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, historydf.values.max()))
    plt.title('Loss: %.3f' % history.history['loss'][-1])
    
def plot_compare_histories(history_list, name_list, plot_accuracy=True):
    dflist = []
    min_epoch = len(history_list[0].epoch)
    losses = []
    for history in history_list:
        h = {key: val for key, val in history.history.items() if not key.startswith('val_')}
        dflist.append(pd.DataFrame(h, index=history.epoch))
        min_epoch = min(min_epoch, len(history.epoch))
        losses.append(h['loss'][-1])

    historydf = pd.concat(dflist, axis=1)

    metrics = dflist[0].columns
    idx = pd.MultiIndex.from_product([name_list, metrics], names=['model', 'metric'])
    historydf.columns = idx
    
    plt.figure(figsize=(6, 8))

    ax = plt.subplot(211)
    historydf.xs('loss', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
    plt.title("Training Loss: " + ' vs '.join([str(round(x, 3)) for x in losses]))
    
    if plot_accuracy:
        ax = plt.subplot(212)
        historydf.xs('acc', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
        plt.title("Accuracy")
        plt.xlabel("Epochs")
    
    plt.xlim(0, min_epoch-1)
    plt.tight_layout()

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print(x_train.shape)
print(x_test.shape)

In [ ]:
# Neuaral Network Construction
# Compare linear and ReLU activation functions. Note that ReLU learns fatser than Sigmoid 
# because sigmoid has flat points on the left and right sides. Experimentally, you can see 
# how bad sigmoid is by changing all AF's to Sigmoid.
input_size = 784
hidden_size = 16
code_size = 32

input_img = Input(shape=(input_size,))
hidden_1_a = Dense(hidden_size, activation='linear')(input_img)
code_a = Dense(code_size, activation='linear')(hidden_1_a)
hidden_2_a = Dense(hidden_size, activation='linear')(code_a)
output_img_a = Dense(input_size, activation='sigmoid')(hidden_2_a)

autoencoder_a = Model(input_img, output_img_a)
autoencoder_a.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder_a.fit(x_train, x_train, epochs=5)


input_img = Input(shape=(input_size,))
hidden_1_b = Dense(hidden_size, activation='relu')(input_img)
code_b = Dense(code_size, activation='relu')(hidden_1_b)
hidden_2_b = Dense(hidden_size, activation='relu')(code_b)
output_img_b = Dense(input_size, activation='sigmoid')(hidden_2_b)

autoencoder_b = Model(input_img, output_img_b)
autoencoder_b.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder_b.fit(x_train, x_train, epochs=5)


In [ ]:
plot_autoencoder_outputs(autoencoder_a, 1, (28, 28))
plot_autoencoder_outputs(autoencoder_b, 1, (28, 28))